In [5]:
# Célula 1: Clonar o Repositório GitHub e Mudar para o Diretório do Projeto
import os
import shutil  # Operações de cópia e movimentação de arquivos
import subprocess  # Execução de comandos externos
import sys  # Acesso ao sistema e argumentos da linha de comando
import time
import traceback  # Exibição detalhada de erros e exceções

import numpy as np  # Operações matemáticas e manipulação de arrays
import torch  # Processamento e inferência com modelos PyTorch
from PIL import Image  # Manipulação de imagens
from ultralytics import YOLO  # Uso do modelo YOLO para detecção de objetos

print("--- Configurando ambiente Google Colab com GitHub ---")

# --- PARÂMETROS DO SEU REPOSITÓRIO ---
# URL do seu repositório GitHub
GIT_REPO_URL = 'https://github.com/kralluz/trabalho_inteligencia_artificial.git'

# Nome da pasta do repositório após o clone (geralmente o nome do repositório)
REPO_NAME = 'trabalho_inteligencia_artificial'
# -----------------------------------

# Clonar o repositório
print(f"Clonando repositório: {GIT_REPO_URL}...")
if os.path.exists(REPO_NAME):
    print(f"Diretório '{REPO_NAME}' já existe. Removendo para clonar novamente...")
    !rm -rf {REPO_NAME}
!git clone {GIT_REPO_URL}

# Mudar o diretório de trabalho para a raiz do repositório clonado
# Isso é crucial para que os caminhos relativos no seu script funcionem
os.chdir(REPO_NAME)
print(f"Diretório de trabalho atual: {os.getcwd()}")

print("\nVerificando conteúdo do diretório do projeto (primeiros 5 itens):")
try:
    listed_items = os.listdir('.')
    for i, item in enumerate(listed_items[:5]):
        print(f"- {item}")
    if len(listed_items) > 5:
        print(f"... e mais {len(listed_items) - 5} itens.")
    elif not listed_items:
        print("O diretório do projeto parece estar vazio após o clone.")
except Exception as e:
    print(f"Erro ao listar o diretório: {e}")

print("\n--- Configuração inicial e clone do GitHub concluídos ---")

--- Configurando ambiente Google Colab com GitHub ---
Clonando repositório: https://github.com/kralluz/trabalho_inteligencia_artificial.git...
Cloning into 'trabalho_inteligencia_artificial'...
remote: Enumerating objects: 804, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 804 (delta 0), reused 1 (delta 0), pack-reused 801 (from 2)
Receiving objects: 100% (804/804), 325.45 MiB | 28.99 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (210/210), done.
Diretório de trabalho atual: /content/trabalho_inteligencia_artificial

Verificando conteúdo do diretório do projeto (primeiros 5 itens):
- projeto_final_novo
- .git
- imagens_para_inferencia
- yolov8n.pt
- dataset_novo
... e mais 5 itens.

--- Configuração inicial e clone do GitHub concluídos ---


In [6]:
# Célula 2: Instalação de Dependências
print("--- [1/6] Instalando dependências... ---")

deps = [
    "pip install ultralytics==8.3.153 --quiet",
    "pip install Pillow numpy matplotlib --quiet"
]

for dep in deps:
    try:
        print(f"Executando: {dep}")
        # Usamos '!' para executar comandos de shell diretamente no Colab
        !{dep}
        print(f"✓ {dep.split()[2]} instalado/verificado.")
    except Exception as e:
        print(f"✗ Erro ao instalar {dep.split()[2]}: {e}")
        print(f"~ {dep.split()[2]} (pode já estar instalado ou houve outro problema).")

print("\n--- Instalação de dependências concluída ---")

--- [1/6] Instalando dependências... ---
Executando: pip install ultralytics==8.3.153 --quiet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.5 MB/s eta 0:00:00
✓ ultralytics==8.3.153 instalado

In [7]:
# Célula 3: Definição da Função 'configurar_dataset_novo()'

print("--- Definindo a função 'configurar_dataset_novo()' ---")

def configurar_dataset_novo():
    """Configura o dataset YOLO novo (convertido com alta qualidade)"""
    print("🚗 Configurando dataset YOLO novo (alta qualidade)...")

    base_path = os.getcwd()
    dataset_path = os.path.join(base_path, "dataset_yolo_novo")
    yaml_path = os.path.join(dataset_path, "data.yaml")

    # Verificar se o dataset existe
    if not os.path.exists(dataset_path):
        print("❌ Dataset YOLO novo não encontrado!")
        print("💡 Certifique-se de que 'dataset_yolo_novo' está na raiz do seu repositório GitHub.")
        return None

    if not os.path.exists(yaml_path):
        print("❌ Arquivo data.yaml não encontrado!")
        print(f"💡 Certifique-se de que '{yaml_path}' existe no seu repositório GitHub.")
        return None

    print(f"✅ Dataset YOLO novo configurado: {dataset_path}")
    print(f"📄 Usando configuração: {yaml_path}")

    # Verificar estrutura do dataset
    total_annotations = 0
    for split in ['train', 'val', 'test']:
        img_dir = os.path.join(dataset_path, split, 'images')
        lbl_dir = os.path.join(dataset_path, split, 'labels')

        if os.path.exists(img_dir) and os.path.exists(lbl_dir):
            img_count = len([f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            lbl_count = len([f for f in os.listdir(lbl_dir) if f.lower().endswith('.txt')])

            # Contar anotações
            for lbl_file in os.listdir(lbl_dir):
                if lbl_file.lower().endswith('.txt'):
                    lbl_path = os.path.join(lbl_dir, lbl_file)
                    with open(lbl_path, 'r') as f:
                        total_annotations += len(f.readlines())

            print(f"  📂 {split}: {img_count} imagens, {lbl_count} labels")
        else:
            print(f"  ⚠️ {split}: diretório '{split}' não encontrado ou incompleto em {dataset_path}")

    print(f"📊 Total de anotações: {total_annotations}")
    return yaml_path

print("\n--- Função 'configurar_dataset_novo()' definida ---")


--- Definindo a função 'configurar_dataset_novo()' ---

--- Função 'configurar_dataset_novo()' definida ---


In [9]:
# Célula 4: Início da Função 'main()' - Importações e Configuração do Dataset

# Bloco de inicialização da função main()
print("=" * 70)
print("TRABALHO FINAL - INTELIGENCIA ARTIFICIAL")
print("Detecção de Vagas de Estacionamento com YOLO")
print("VERSÃO FINAL - DATASET NOVO DE ALTA QUALIDADE")
print("903 anotações precisas em 30 imagens")
print("=" * 70)

start_time = time.time() # Inicia a contagem de tempo

print("\n[2/6] Importando bibliotecas...")
try:
    # As importações já estão no topo da célula
    print("✓ Todas as bibliotecas importadas")
except ImportError as e:
    print(f"✗ Erro de importação: {e}")
    # sys.exit(1) # Pode adicionar um sys.exit(1) para parar a execução em caso de erro fatal

print("\n[3/6] Configurando dataset YOLO novo (alta qualidade)...")
try:
    # Chama a função definida na Célula 3
    yaml_path = configurar_dataset_novo()
    if yaml_path:
        print("✅ Dataset YOLO novo configurado com sucesso")
    else:
        print("❌ Erro ao configurar dataset YOLO novo")
        # sys.exit(1) # Pode adicionar um sys.exit(1) para parar a execução em caso de erro fatal
except Exception as e:
    print(f"✗ Erro ao configurar dataset: {e}")
    # sys.exit(1) # Pode adicionar um sys.exit(1) para parar a execução em caso de erro fatal

print("\n[4/6] Carregando modelo YOLO...")
try:
    # Tentar carregar modelo pré-treinado
    try:
        model = YOLO('yolov8n.pt')
        print("✓ Modelo pré-treinado yolov8n.pt carregado")
    except Exception as e1:
        print(f"~ Não foi possível carregar yolov8n.pt ({e1}), tentando yolov8n.yaml...")
        try:
            model = YOLO('yolov8n.yaml')
            print("✓ Modelo criado do zero a partir de yolov8n.yaml")
        except Exception as e2:
            print(f"✗ Erro ao criar modelo com yolov8n.yaml: {e2}")
            # sys.exit(1) # Pode adicionar um sys.exit(1)
except Exception as e:
    print(f"✗ Erro fatal ao carregar/criar modelo: {e}")
    # sys.exit(1) # Pode adicionar um sys.exit(1)

# As variáveis 'model' e 'yaml_path' precisam ser acessíveis nas próximas células.
# Em notebooks, variáveis definidas em uma célula são acessíveis nas seguintes.
print("\n--- Importações e Configuração do Dataset concluídas ---")

TRABALHO FINAL - INTELIGENCIA ARTIFICIAL
Detecção de Vagas de Estacionamento com YOLO
VERSÃO FINAL - DATASET NOVO DE ALTA QUALIDADE
903 anotações precisas em 30 imagens

[2/6] Importando bibliotecas...
✓ Todas as bibliotecas importadas

[3/6] Configurando dataset YOLO novo (alta qualidade)...
🚗 Configurando dataset YOLO novo (alta qualidade)...
✅ Dataset YOLO novo configurado: /content/trabalho_inteligencia_artificial/dataset_yolo_novo
📄 Usando configuração: /content/trabalho_inteligencia_artificial/dataset_yolo_novo/data.yaml
  📂 train: 21 imagens, 21 labels
  📂 val: 4 imagens, 4 labels
  📂 test: 5 imagens, 5 labels
📊 Total de anotações: 903
✅ Dataset YOLO novo configurado com sucesso

[4/6] Carregando modelo YOLO...
✓ Modelo pré-treinado yolov8n.pt carregado

--- Importações e Configuração do Dataset concluídas ---


In [ ]:
# Célula 5: Treinamento do Modelo
print("--- [5/6] Treinando modelo com dataset de alta qualidade... ---")

try:
    print("Iniciando treinamento (pode demorar alguns minutos)...")
    print("📊 Dataset: 903 anotações precisas de vagas!")

    # Limpar projetos anteriores
    if os.path.exists('projeto_final_novo'):
        print("Removendo treinamento anterior...")
        shutil.rmtree('projeto_final_novo')

    # Configurações de treinamento otimizadas para o novo dataset
    results = model.train( # 'model' e 'yaml_path' são acessíveis da Célula 4
        data=yaml_path,
        epochs=150,           # Menos épocas pois o dataset é de alta qualidade
        batch=8,              # Batch maior pois temos menos imagens mas mais anotações
        imgsz=640,            # Resolução maior para melhor precisão
        device='cpu',         # Mantenha 'cpu' ou mude para '0'/'cuda' se for usar GPU
        project='projeto_final_novo',
        name='yolo_vagas_novo',
        exist_ok=True,
        pretrained=True,
        optimizer='AdamW',    # Otimizador mais moderno
        verbose=True,
        seed=42,
        deterministic=True,
        patience=10,          # Early stopping mais agressivo
        save_period=5,        # Salvar checkpoints a cada 5 épocas
        val=True,
        cache=False,
        lr0=0.01,             # Learning rate inicial
        warmup_epochs=3       # Warmup epochs
    )
    print(f"✓ Treinamento concluído! Resultados salvos em: {results.save_dir}")

except Exception as e:
    print(f"✗ Erro no treinamento: {e}")
    print("Detalhes do erro:")
    traceback.print_exc()
    print("Continuando com validação...")

print("\n--- Treinamento do modelo concluído ---")

--- [5/6] Treinando modelo com dataset de alta qualidade... ---
Iniciando treinamento (pode demorar alguns minutos)...
📊 Dataset: 903 anotações precisas de vagas!
Removendo treinamento anterior...
New https://pypi.org/project/ultralytics/8.3.154 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/trabalho_inteligencia_artificial/dataset_yolo_novo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7

train: Scanning /content/trabalho_inteligencia_artificial/dataset_yolo_novo/train/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1619.9±676.1 MB/s, size: 389.1 KB)



val: Scanning /content/trabalho_inteligencia_artificial/dataset_yolo_novo/val/labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

Plotting labels to projeto_final_novo/yolo_vagas_novo/labels.jpg... 


optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to projeto_final_novo/yolo_vagas_novo
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150         0G      1.462      3.404      1.348        333        640:  33%|███▎      | 1/3 [00:08<00:17,  8.87s/it]

In [ ]:
# Célula 6: Testes e Inferência Final
print("--- [6/6] Testando modelo e realizando inferência final... ---")

model_path_best = os.path.join('projeto_final_novo', 'yolo_vagas_novo', 'weights', 'best.pt')
model_path_last = os.path.join('projeto_final_novo', 'yolo_vagas_novo', 'weights', 'last.pt')

loaded_model = None
if os.path.exists(model_path_best):
    try:
        loaded_model = YOLO(model_path_best)
        print(f"✓ Modelo treinado carregado: {model_path_best}")
    except Exception as e_best:
        print(f"~ Problema ao carregar best.pt ({e_best}), tentando last.pt...")
        if os.path.exists(model_path_last):
            try:
                loaded_model = YOLO(model_path_last)
                print(f"✓ Modelo treinado carregado: {model_path_last}")
            except Exception as e_last:
                print(f"~ Erro ao carregar last.pt ({e_last}). Teste abortado.")
elif os.path.exists(model_path_last):
    try:
        loaded_model = YOLO(model_path_last)
        print(f"✓ Modelo treinado carregado: {model_path_last}")
    except Exception as e_last:
        print(f"~ Erro ao carregar last.pt ({e_last}). Teste abortado.")
else:
    print("~ Nenhum modelo treinado encontrado. Teste abortado.")

if loaded_model:
    try:
        # Diretório de teste
        test_images_dir = os.path.join(os.path.dirname(yaml_path), 'test', 'images')

        # Criar diretório para resultados da inferência
        results_dir = "resultados_novo/predicoes_finais"
        os.makedirs(results_dir, exist_ok=True)

        print(f"Realizando inferência nas imagens de teste em: {test_images_dir}")
        print(f"Resultados serão salvos em: {results_dir}")

        results_pred = loaded_model.predict(
            source=test_images_dir,
            save=True,
            project="resultados_novo",
            name="predicoes_finais",
            exist_ok=True,
            conf=0.1,   # Confiança mais baixa para capturar mais detecções
            iou=0.5,    # IoU para NMS
            verbose=True
        )

        # Analisar resultados detalhadamente
        total_detections = 0
        for i, result in enumerate(results_pred):
            if result.boxes is not None and len(result.boxes) > 0:
                detections = len(result.boxes)
                total_detections += detections
                print(f"  📸 Imagem {i+1}: {detections} vagas detectadas")

                if i < 3:  # Mostrar detalhes das 3 primeiras imagens
                    for j, box in enumerate(result.boxes[:5]):  # Max 5 detecções por imagem
                        cls = int(box.cls[0])
                        conf = float(box.conf[0])
                        class_name = loaded_model.names[cls]
                        print(f"    - Vaga {j+1}: {class_name} (confiança: {conf:.2f})")
            else:
                print(f"  📸 Imagem {i+1}: 0 vagas detectadas")

        print(f"✓ Teste concluído! {len(results_pred)} imagens processadas.")
        print(f"📊 Total de detecções: {total_detections}")

    except Exception as e:
        print(f"~ Erro no teste: {e}")
        traceback.print_exc()
else:
    print("~ Modelo não carregado, pulando etapa de teste.")

# Teste final com imagens para inferência
print("\n" + "=" * 60)
print("🚗 TESTE FINAL COM IMAGENS DE INFERÊNCIA")
print("=" * 60)

inference_images_path = os.path.join(os.getcwd(), 'imagens_para_inferencia')

if loaded_model and os.path.exists(inference_images_path):
    try:
        print(f"Realizando inferência final com dataset de alta qualidade em: {inference_images_path}")

        inference_results = loaded_model.predict(
            source=inference_images_path,
            save=True,
            project='teste_final_novo',
            name='inferencia_final',
            exist_ok=True,
            conf=0.1,   # Confiança baixa para capturar mais detecções
            iou=0.5,
            verbose=True
        )

        # Analisar resultados da inferência final
        total_detections = 0
        for result in inference_results:
            if result.boxes is not None:
                detections = len(result.boxes)
                total_detections += detections

                # Analisar por classe
                free_count = 0
                occupied_count = 0

                for box in result.boxes:
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])

                    if loaded_model.names[cls] == 'free_parking_space':
                        free_count += 1
                    elif loaded_model.names[cls] == 'not_free_parking_space':
                        occupied_count += 1

                print(f"  📸 {os.path.basename(result.path)}: {detections} vagas")
                print(f"    🟢 Livres: {free_count} | 🔴 Ocupadas: {occupied_count}")
            else:
                print(f"  📸 {os.path.basename(result.path)}: 0 vagas detectadas")

        print(f"\n✓ Inferência final concluída! {len(inference_results)} imagens processadas.")
        print(f"📁 Resultados salvos em: teste_final_novo/inferencia_final/")
        print(f"📊 Total de detecções: {total_detections}")

    except Exception as e:
        print(f"⚠️ Erro na inferência final: {e}")
        traceback.print_exc()
else:
    print("~ Diretório 'imagens_para_inferencia' não encontrado ou modelo não carregado. Pulando inferência final.")

# Resumo final
end_time = time.time() # A variável start_time é do início da Célula 4
duration = (end_time - start_time) / 60

print("\n" + "=" * 70)
print("PROJETO CONCLUÍDO COM DATASET DE ALTA QUALIDADE!")
print("=" * 70)
print(f"Tempo total: {duration:.1f} minutos")

if os.path.exists('projeto_final_novo'):
    print("\nArquivos criados:")
    print("✓ dataset_yolo_novo/ - Dataset de alta qualidade (903 anotações)")
    print("✓ projeto_final_novo/ - Modelo treinado")
    if os.path.exists('resultados_novo'):
        print("✓ resultados_novo/ - Predições do teste")
    if os.path.exists('teste_final_novo'):
        print("✓ teste_final_novo/ - Inferência final")

    print("\nEste projeto demonstra:")
    print("• Uso de dataset de ALTA QUALIDADE com anotações manuais precisas")
    print("• 903 anotações de vagas em 30 imagens (30 vagas/imagem)")
    print("• Labels profissionais baseados em polígonos convertidos para YOLO")
    print("• Treinamento otimizado para dataset de qualidade superior")
    print("• Detecção precisa de vagas livres e ocupadas")

    print("\n🎉 SUCESSO TOTAL COM DATASET NOVO DE ALTA QUALIDADE! ✓")
else:
    print("Projeto executado com limitações")

print("\n--- Testes e Inferência Final concluídos ---")